In [5]:
using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs

myInstance = "2"

outfile = "/Users/din/Desktop/Research_S2019/Outputs/Base_NoRep_CVaR_"*myInstance*".txt"
gurobi_env = Gurobi.Env()

to = TimerOutput()
myFile = "myData"*myInstance*".jl"
include(myFile)

#println(last_node)
#println(edge)
#println("cL_orig = ", cL_orig)
#println("cU_orig = ", cU_orig)
#println("d = ",d)
#=
seqnum = collect(1:(last_node-1))
incoming = 1 .* (edge[1:(end-1),2] .== seqnum)
leaving = 1 .* (edge[1:(end-1),1] .== seqnum)

println("Incoming")
println("New array:", incoming)

h = Model(solver = GurobiSolver(Heuristics=0.0, Cuts = 0, OutputFlag = 0)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 
=#
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)

Academic license - for non-commercial use only


In [6]:
#SETTING THE INITIAL MODEL FOR H
            #=TEMPORARY INFO:
            c_L = cL_orig
            c = 0.5*(cL_orig + cU_orig)
            M = cU_orig - cL_orig
            x_now = [0 0 1]
            ##END OF TEMPORARY INFO =====#

h1 = Model(solver = GurobiSolver(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 

@variable(h1, 1 >= y_h[1:Len]>=0)
#@variable(h, q[1:Len]>=0)

#Setting constraint for start node
leaving = 1 .* (edge[j=1:Len,1] .== origin)
@constraint(h1, sum(y_h[k]*leaving[k] for k=1:Len) == 1)

#Setting constraint for other nodes
for i in all_nodes
    if i != destination
        if i != origin
            incoming = 1 .* (edge[j=1:Len,2] .== i)
            leaving = -1 .* (edge[j=1:Len,1] .== i)
            @constraint(h1, sum(y_h[k]*leaving[k] + y_h[k]*incoming[k] for k=1:Len) == 0)
        end
    end
end
#@objective(h, Min, sum((cL_orig[i]+d[i]*x_now[i])*y_h[i] + q[i] for i=1:Len))
#print(h1)
#solve(h1)


function gx_bound(c_L, c_U, c, c_g, x_now, edge)
    
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    start_node = edge[:,1]
    end_node = edge[:,2]

    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)


    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = Graph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end

    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
    #parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    #println(f,"y vector:", y)
    
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
    T = Int64[]
    for i = 1:Len
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end

    y_index = find(y .== 1)

    #println("Minimum Spanning Tree: ", T)
    #println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
    #println("Minimum Spanning Tree =", pred)
    #println("Node label =" ,label)

    return y, gx, SP
end


function hx_bound(h, c_L, c_U, c, d, x_now, edge)
    
    c = (c_L + c_U)/2
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f,"Current interdiction x = ", x_now)
    M = c_U - c_L

    h2 = copy(h1)

    y2 = h2[:y_h]

    @variable(h2, q[1:Len]>=0)
    for i = 1:Len
        @constraint(h2, q[i] >= c[i] - c_L[:,1][i] - M[i]*(1-y2[i])) #_h[i]))
    end

    @objective(h2, Min, sum((c_L[i]+d[i]*x_now[i])*y2[i] + q[i] for i=1:Len))

    #print(f,h2)
    solve(h2)
    hx = getobjectivevalue(h2)
    
    return getvalue(y2), hx
end

f = open(outfile, "w")
#MAIN PROGRAM:
df = DataFrame(CELL = Int[],ARCS_USED = Array[], LB = Array[], UB = Array[], PROB = Float64[], SP = Float64[])
df2 = DataFrame(ITER = Int[], X = Array[])
Cell_List = [1]
#=
weighted_unc = []
w_u = sum(cU_orig[i] - cL_orig[i] for i = 1:Len)
push!(weighted_unc, w_u)
weighted_unc_temp = copy(weighted_unc)
=#
last_x = zeros(Len)
checkedCells = Int64[]

MP_obj = 0.0

push!(df, (1, yy, cL_orig, cU_orig, 1, SP_init))

##println(f,"MASTER PROBLEM==========================================================================================")

@timeit to "MP" begin 
m = Model(solver = GurobiSolver(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 

@variable(m, w )
@variable(m,  x[1:Len], Bin)
@variable(m, 1e6 >= z[1:10000] >= 0)
@constraintref constr[1:10000]
@constraint(m, sum(x[i] for i=1:Len) <= 2.0) #attack budget = 2
constr[1] = @constraint(m, w <= SP_init + sum(yy[i]*x[i]*d[i] for i=1:Len) + z[1]  )
@objective(m, Max, α*w - sum(p[i]*z[i] for i = 1:length(p)) )#w - sum(s[k] for k=1:length(s))/length(s) )
##println(f,p)
##println(f,z[1:length(p)])

con_num = 1
stopping_cond = delta2 + 0
total_time = 0.0
iter = 0
@timeit to "BIGWHILE" while stopping_cond >= delta2 
        tic()
        stopping_cond = 0
        iter = iter + 1
#         println(f,"\n\nITERATION ",length(df[:CELL]),": ========================================================================================== ")
#         println(f, "Total # cells ", length(Cell_List))
        #println(f, m)
        @timeit to "SOLVE M" begin
            solve(m) 
            end# END OF SOLVE M TIMING BLOCK
            
            cell_num = maximum(df[:CELL])
            MP = getvalue(α*w - sum(p[i]*z[i] for i = 1:length(p)))
            MP_obj = copy(MP)
            x_now = getvalue(x) + 0.0
            
            if x_now != last_x 
                checkedCells = Int64[]
            end
            last_x = x_now
        
#             println(f,"Interdiction ", x_now)
            push!(df2, (length(df[:CELL]), x_now))
        
            z_now = getvalue(z)+ 0.0 
            ##println(f,"z = ", z_now[1:length(p)])

            c_L = cL_orig + 0.0
            c_U = cU_orig + 0.0
            c = (c_L + c_U)/2
            y = zeros(length(c_L))
            r = 0
            r2 = 0
            gap = 0.0
            
            t = 0
            k = 0
            found = false
            
        #for k = 1: length(p)
        myLength = length(p)
        while found == false && k < myLength #length(p)# k = 1: cell_num #length(p)
            k=k+1
            if (k in checkedCells) == false
                @timeit to "Pre-gx" begin

                    if length(find(df[:CELL] .== k))>0
                        row = find(df[:CELL] .== k)[1]
                        c_L = df[:LB][row] 
                        c_U = df[:UB][row] 
                    else
                        c_L = cL_orig[:,1] 
                        c_U = cU_orig[:,1] 
                    end
                    c = (c_U + c_L)/2
                    M = zeros(Len)
                    for i = 1:Len
                        M[i] = c_U[i] - c_L[i]
                    end
                    c_g = c + d.*x_now
                    end # END OF Pre-gx TIMING BLOCK



                ##println(f,"==================SUBPROBLEM 1: G(X), CURRENT CELL = ", k)
#                 println(f, "Current Cell = ", k)
                @timeit to "gx" begin
                    y, gx, SP = gx_bound(c_L, c_U, c, c_g, x_now, edge)
                    ##println(f,"z[k] = ", getvalue(z[k]))
                    ##println(f,"gx[k] = ", gx)

                    if length(g) >= k 
                        g[k] = gx
                    else
                        g = push!(g,gx)
                    end  
                    end #END OF GX TIMING BLOCK

                if (z_now[k] + gx) < getvalue(w) - delta1   
                    @timeit to "Add Constraints (1)" begin
                        stopping_cond = delta2 + 1
                        r = k 
                        #println(f,"k = ", k)
                        push!(df, (r, y, c_L + zeros(length(c_L)), c_U + zeros(length(c_U)), p[r], SP))

                        last_row = length(df[:CELL])
                        con_num = con_num + 1 
                        constr[last_row] = @constraint(m, w <= z[r] + sum(c[i]*y[i] + d[i]*y[i]*x[i] for i = 1:length(c_L)))
                        ##println(f,"Constraint #", last_row," is :", constr[last_row])
                        @objective(m, Max, α*w - sum(p[i]*z[i] for i = 1:length(p)) )

                        found = true
                        ##println(f,"FOUND = ", found)
                        ##println(f,"FOR CELL ", k, ", STOPPING CONDITION (after gx): ", stopping_cond)
                        end #END OF CONSTRAINT 1 TIMING BLOCK

#                         println(f,"z = ", z_now[k])
#                         println(f,"g = ", gx)
                else

                    ##println(f,"==================SUBPROBLEM 2: H(X), CURRENT CELL = ", k)
                    @timeit to "hx" begin
                        y_h, hx = hx_bound(h, c_L, c_U, c, d, x_now, edge)
                        ##println(f,"gx = ", gx)
                        ##println(f,"hx = ", hx)
                        ##println(f,"k = ", k)
                        if length(h) >= k 
                            h[k] = hx
                        else
                            h = push!(h,hx)
                        end
                        end #END OF HX TIMING BLOCK

#                         println(f,"g = ", gx)
#                         println(f,"h = ", hx)

                    if gx-hx > delta2
                        @timeit to "Add Constraints (2)" begin    
                            stopping_cond = delta2 + 1
                            found = true
                            arc_split = find(M .== maximum(M))[1]
                            gap = (c_U[arc_split] - c[arc_split])/2
                            r2 = maximum(df[:CELL])+1
                            #weighted_unc[k] = weighted_unc[k] - gap*(p[k]/2)
                            #push!(weighted_unc, weighted_unc[k])

                            #weighted_unc_temp[k] = weighted_unc[k]
                            #push!(weighted_unc_temp, weighted_unc_temp[k])


                            push!(df, (r2, y, df[:LB][row]+zeros(length(c_L)), df[:UB][row]+zeros(length(c_L)), p[k]/2, SP))
                            push!(p,p[k]/2)
                            push!(Cell_List, r2)


                            p[k]=p[k]/2
                            last_row = length(df[:CELL])
                            df[:LB][last_row][arc_split] = c[arc_split] + 0.0
                            c_temp = (df[:UB][last_row] + df[:LB][last_row]) / 2
                            y_temp = df[:ARCS_USED][last_row]
                            SP = sum(c_temp[t]*y_temp[t] for t=1:Len)
                            df[:SP][last_row] = SP

                            for i = 1: length(df[:CELL])-1
                                if df[:CELL][i] == k 
                                    df[:PROB][i] = df[:PROB][i]/2
                                    df[:UB][i][arc_split] = c[arc_split]+ 0.0

                                    #RE-CALCULATING COST OF COPIED SHORTEST PATH
                                    c_temp = (df[:UB][i] + df[:LB][i]) / 2
                                    y_temp = df[:ARCS_USED][i]
                                    SP = sum(c_temp[t]*y_temp[t] for t=1:Len)
                                    df[:SP][i] = SP
                                    JuMP.setRHS(constr[i], df[:SP][i])
                                end
                            end
                            constr[last_row] = @constraint(m, w - z[df[:CELL][last_row]] <=   sum(d[i]*x[i]*y[i] for i = 1:length(c_L)) + df[:SP][last_row]) 
                            @objective(m, Max, α*w - sum(p[i]*z[i] for i = 1:length(p)) )
                            end #END OF CONSTRAINT 2 TIMING BLOCK
                    else
                        push!(checkedCells, k)
                    end
                    ##println(f,"FOUND = ", found)
                    ##println(f,"FOR CELL ", k, ", STOPPING CONDITION (after adding constraints) : ", stopping_cond)
                end
            end
        #@timeit to "WRITE TO FILE" CSV.write("/Users/din/Desktop/Research/OUTPUT(Test5Nodes_Mod).cvs", df)  
        end
    
        time_lapse = toq()    
        ##println(f, "Elapsed Time for Iteration ", k," = ", time_lapse)
        total_time = total_time + time_lapse
        ##println(f,"Total Running Time = ", total_time)
        #=if total_time > 100
            break
        end=#
    end
    
    end #END OF MP TIMING BLOCK    

##CSV.write("/Users/din/Desktop/Research/Cell_Ins1_BaseClever.cvs", df)  
##CSV.write("/Users/din/Desktop/Research/InterdicionX_Ins1_BaseClever", df2) 

@timeit to "CALC OPT GAP" opt_gap = sum(p[i]*(g[i] - h[i]) for i = 1:length(p))
##println(f,"\n\n\nFinal z = ", getvalue(z[1:length(p)]))
##println(f,"Final g = ", g)
##println(f,"Final h = ", h)
##println(f,"List of all cells = ", Cell_List)

println(f,"Optimality gap = ", opt_gap)
println(f,"MP_obj = ", MP_obj)
println(f,"Overall runtime = ", total_time)
println(f,"No. iterations = ", iter) #length(df[:CELL]))
# for i = 1 : length(g)
#     println(f, g[i],"\t", h[i])
# end
    
close(f)

show(to; allocations = false)

 ────────────────────────────────────────────────────────
                                           Time          
                                   ──────────────────────
         Tot / % measured:              6.08s / 89.4%    

 Section                   ncalls     time   %tot     avg
 ────────────────────────────────────────────────────────
 MP                             1    5.41s  99.4%   5.41s
   BIGWHILE                     1    5.38s  99.0%   5.38s
     SOLVE M                  232    4.61s  84.8%  19.9ms
     hx                       330    283ms  5.20%   857μs
     gx                       421    232ms  4.27%   551μs
     Add Constraints (2)      140   78.6ms  1.44%   561μs
     Pre-gx                   421   49.4ms  0.91%   117μs
     Add Constraints (1)       91   6.37ms  0.12%  70.0μs
 CALC OPT GAP                   1   33.4ms  0.61%  33.4ms
 ────────────────────────────────────────────────────────